**Número de grupo:** 25

**Nombre de los integrantes del grupo:**
- David Bugoi
- Daniela Alejanda Córdova
- Erik Karlgren Domercq
 
Lab 11

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (<i>least common subsummer</i>).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [ ]:
import json
filename = "query.json"
with open(filename, encoding='utf-8') as f:
    data = json.load(f)

dicNombres = {}
dicNombresInversa ={}
dicSubclases = {}
dicSuperclases = {}

for i in data:
    etiquetaX = i['x'].replace('http://www.wikidata.org/entity/', '')
    etiquetaY = i['y'].replace('http://www.wikidata.org/entity/', '')
    dicNombres[i['xLabel']] = etiquetaX
    dicNombresInversa[etiquetaX]=i['xLabel']
    dicNombres[i['yLabel']] = etiquetaY
    dicNombresInversa[etiquetaY]=i['yLabel']

    if dicSubclases.__contains__(etiquetaY):
        dicSubclases[etiquetaY].append(etiquetaX)
    else:
        dicSubclases[etiquetaY] = [etiquetaX]
    if dicSuperclases.__contains__(etiquetaX):
        dicSuperclases[etiquetaX].append(etiquetaY)
    else:
        dicSuperclases[etiquetaX] = [etiquetaY]

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [ ]:
from collections import deque 
def _print_tree_(name, prof):
    etiqueta = dicNombres[name]
    i = 0
    pila = []
    pila.append((i,etiqueta))
    
    while pila:
        i, actual = pila.pop()
        strigFinal = str(i) + " " + str(dicNombresInversa[actual]) + " (" + actual + ")"
        string_length = len(strigFinal) + i  # will be adding 10 extra spaces
        string_revised = strigFinal.rjust(string_length)
        print(string_revised)
        
        if dicSubclases.__contains__(actual):
            for key in dicSubclases[actual]:
                if(i+1<prof):
                    pila.append((i+1,key))


In [ ]:
_print_tree_("guitar", 4)

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```python
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603'}
```

In [ ]:
def todosConceptosGenerales(e):
    sol = set()
    m = dicSuperclases[dicNombres[e]]
    pila = []
    i=0
    for key in m:
        pila.append((i, key))
    while pila:
        i, actual = pila.pop()
        if actual not in sol:
            sol.add(actual)
        if dicSuperclases.__contains__(actual):
            for key in dicSuperclases[actual]:
                pila.append((i+1,key))
    return sol

In [ ]:
todosConceptosGenerales("Brahms guitar")

In [ ]:
dicNombresInversa["Q1798603"]

In [ ]:
def _LCS_(A, B):
    return 'Q34379'

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profundidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```python
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [ ]:
def key_to_string(key):
    return f'{dicNombresInversa[key]} ({key})'

# Busca el camino mínimo entre los conceptos A y B
def path(a,b):
    found = False            # indica si hemos encontrado 'b'
    nodes_to_visit = deque() # lista con los nodos por visitar
    previous = dict()        # diccionario con el nodo (o clave) anterior a cada nodo (o clave)
    
    nodes_to_visit.append(a) # primero visitamos el concepto 'a'
    previous[a] = ''
    
    # búsqueda en profundidad
    while nodes_to_visit and not found:
        current_node = nodes_to_visit.popleft()
        
        if current_node == b:
            found = True
            
        elif dicSubclases.__contains__(current_node):
            for item in dicSubclases[current_node]:
                nodes_to_visit.appendleft(item)
                previous[item] = current_node
    
    # reconstruimos la solución
    if not found:
        return list()
    else:
        path = deque()
        key = b
        while key != '':
            path.appendleft(key_to_string(key))
            key = previous[key]
        return list(path)
        

In [ ]:
path('Q186506', 'Q54634726')

In [ ]:
path('Q17172850', 'Q6012300')

In [ ]:
path('Q27914', 'Q27914')

In [ ]:
path('Q34379', 'Q55724840')

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```python
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [ ]:
root = 'Q34379' # 'root' es instrumento musical

def similarity(a,b):
    lcs = _LCS_(a,b)
    lcs_to_a_path = path(lcs, a)
    lcs_to_b_path = path(lcs, b)
    root_to_lcs_path = path(root, lcs)
    
    # puede fallar
    sim = (len(root_to_lcs_path) - 1) / (len(root_to_lcs_path) + len(lcs_to_a_path) + len(lcs_to_b_path) - 3)
    
    print(sim)
    print(root_to_lcs_path)
    print(lcs_to_a_path)
    print(lcs_to_b_path)

In [ ]:
similarity('Q6012297', 'Q54634726')

### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```